In [2]:
from __future__ import print_function
import numpy as np 
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np


import lsst.sims.maf.db as db
import lsst.sims.maf.metrics as metrics
import lsst.sims.maf.slicers as slicers
import lsst.sims.maf.metricBundles as metricBundles

## Three visits in 2 filters with specified filter pair and time gap


for Fast transients identification

In [3]:
from lsst.sims.maf.metrics import BaseMetric


In [4]:
class threeVisitsWColorMetric(BaseMetric):
        

    def __init__(self, filterCol='filter', nightCol='night', 
                 mjdCol='observationStartMJD', dt1=0.5/24, dt2=1.5/24,
                 filters='y'+'g', nVisitsMin=3, **kwargs):
        '''
        Calculates fields for which there are 3 visits in 2 filters with color within dt1 
        and shape at least separated by dt2
        
        returns 1 if the conditionas are all verified for the data slice, -666 otherwise
        default: dt1 30 min dt2 1.5 hours
        default filters y+g (cause there are a few)'''
        
        # Pass the same bins to the plotter.                                                      
        self.mjdCol = mjdCol
        self.nightCol = nightCol
        self.filterCol = filterCol        
        self.nVisitsMin = nVisitsMin
        self.dt1 = dt1
        self.dt2 = dt2
        self.filters = (filters[0], filters[1])
        super(threeVisitsWColorMetric, 
              self).__init__(col=[self.mjdCol, 
                                self.nightCol, 
                                self.filterCol], 
                                metricDtype='int', 
                                units='Fraction', 
                                **kwargs)

    def run(self, dataSlice, slicePoint=None):
        mins = [99, 99]
        maxs = [-99, -99]
        
        #count the observations within a night for each slice
        n, counts = np.unique(dataSlice[self.nightCol], return_counts=True)
        #print(dataSlice)
        minobs = n[counts >= 3]
        #print(n, counts, minobs)
        dt1, dt2, filtshape = -1, -1, 'x'
        
        #iterate over nights with more than 3 observations
        for i,night in enumerate(minobs):
            #this is a night with >=3 observations
            good = dataSlice[self.nightCol] == night
            
            #check if it has 2 filters at least
            n, count = np.unique(dataSlice[self.filterCol][good], return_counts=True)
            if len(n) > 1:
                #print(n, count)
                #obs with filter 1
                f0 = dataSlice[self.filterCol][good] == self.filters[0]
                #obs with filter 2
                f1 = dataSlice[self.filterCol][good] == self.filters[1]
                
                #filter 1 is shape
                if f0.sum() > 1 and f1.sum() == 1:
                    #max gap with same filter
                    dt2 = np.max(np.abs(np.diff(dataSlice[self.mjdCol][good][f0])))
                    #min between 2 colors
                    dt1 = np.min(
                        [np.abs(t1-t2) for t1 in dataSlice[self.mjdCol][good][f0] 
                         for t2 in dataSlice[self.mjdCol][good][f1]])
                    filtshape = self.filters[0]
                    #print (dt1, dt2, filtshape)
                    
                #filter 2 is shape
                elif f0.sum() == 1 and f1.sum() > 1:
                    #max gap with same filter
                    dt2 = np.max(np.abs(np.diff(dataSlice[self.mjdCol][good][f0])))
                    #min between 2 colors
                    dt1 = np.min(
                        [np.abs(t1-t2) for t1 in dataSlice[self.mjdCol][good][f0] 
                         for t2 in dataSlice[self.mjdCol][good][f1]])
                    filtshape = self.filters[1]
                    #print (dt1, dt2, filtshape)

                #either filter could be shape
                elif f0.sum() > 1 and f1.sum() > 1:
                    #max gap with same filter
                    dt2_0 = np.max(np.abs(np.diff(dataSlice[self.mjdCol][good][f0])))
                    dt2_1 = np.max(np.abs(np.diff(dataSlice[self.mjdCol][good][f1])))
                    #print(max(dt2_0, dt2_1))
                    dt2 = max(dt2_0, dt2_1)
                    filtshape = np.array([self.filters[0], 
                                          self.filters[1]])[np.array([dt2_0, 
                                                                      dt2_1]) == dt2]
                    #print(filtshape)
                    #min gap between 2 filters gives color
                    dt1 = np.min(
                        [np.abs(t1-t2) for t1 in dataSlice[self.mjdCol][good][f0] 
                         for t2 in dataSlice[self.mjdCol][good][f1]])
            else:
                # no 3 obs night
                metricval = 0.
                return metricval
        #see if obs verify  gap min and max conditions
        if dt1 >  -1 and dt1 <= self.dt1 and dt2 > -1 and dt2 >= self.dt2:
            metricval = 1.0
        else :
            metricval = 0.
        return metricval
    
    def reducep(self, metricval):
        result = np.mean(metricval)
        return result


In [ ]:
metric = threeVisitsWColorMetric8()
slicer = slicers.HealpixSlicer(nside=8)
sqlconstraint = ''
myBundle = metricBundles.MetricBundle(metric, slicer, sqlconstraint)

opsdb = db.OpsimDatabase('baseline2018a.db')
myBundle.setPlotDict({'colorMin': 0.0, 'colorMax': 1.8})
bgroup = metricBundles.MetricBundleGroup(
    {0: myBundle}, opsdb, outDir='newmetric_test', resultsDb=None)
bgroup.runAll()
bgroup.plotAll(closefigs=False)


Healpix slicer using NSIDE=8, approximate resolution 439.742261 arcminutes
Querying database SummaryAllProps with no constraint for columns ['fieldRA', 'filter', 'fieldDec', 'night', 'observationStartMJD'].
